# Dependencies

In [1]:
!pip3 install pyzmq cryptography sympy

Defaulting to user installation because normal site-packages is not writeable


# Initialization

In [13]:
from cryptography.fernet import Fernet
import sympy
import random
import zmq

In [66]:
# math
BITS_OF_PRIME_GROUP = 64

# socket
LOCAL_PORT = 4080
SERVER_HOST = "localhost"
SERVER_PORT = 4080

# Basics

## Primes and Modular arithmetics

In [20]:
def get_random_prime():
    """
    Generate a random prime number of the specified number of bits.
    """

    lower_bound = 2**(BITS_OF_PRIME_GROUP - 1)
    upper_bound = 2**BITS_OF_PRIME_GROUP - 1

    prime = sympy.randprime(lower_bound, upper_bound)
    
    return prime

def find_generator(prime):
    """
    For every prime divisor q of p -1 check if g^{(p-1)/q} ≡ 1 mod p. If that happens, discard g
    If it survives till the last prime divisor of p - 1 is a generator.
    """

    factors = sympy.primefactors(prime - 1)

    while True:
        candidate = random.randint(1, prime - 1)

        for factor in factors:
            if pow(candidate, (prime - 1) // factor, prime) == 1:
                break

        else:
            return candidate

## Secret Key Encryption

The cryptography library contains Fernet, an implementation of symmetric (secret key) authenticated cryptography.

In [54]:
def secret_generate_key():
    """
    Generate a secret key.
    """

    key = Fernet.generate_key()

    return key

def secret_encrypt(msg, key):
    """
    Encrypts a message with a key.
    https://cryptography.io/en/latest/fernet/
    """

    # initialize
    f = Fernet(key)

    # encrypt
    token = f.encrypt(msg)

    return token

def secret_decrypt(token, key):
    """
    Decrypts a message with a key.
    https://cryptography.io/en/latest/fernet/
    """

    # initialize
    f = Fernet(key)

    # decrypt
    msg = f.decrypt(token)

    return msg

In [65]:
test_string = b"Hello, World!"

# generate a random key
key = secret_generate_key()

# encrypt the test string
token = secret_encrypt(test_string, key)

# decrypt the test string
msg = secret_decrypt(token, key)

print("Original message: ", test_string)
print("Encrypted message: ", token)
print("Decrypted message: ", msg)
print("Key: ", key)

Original message:  b'Hello, World!'
Encrypted message:  b'gAAAAABnmmqfERZ4p9qs6SymI2ZL706Weli8ntCoF43FOiDQitd2_D85zQO0f9eFHWPLT2HNh8CWvl9l3Eaiv98Dh8D0FcgSIQ=='
Decrypted message:  b'Hello, World!'
Key:  b'Tmp5-Mq1ZzPGNLXX0mkmPw7OQixXoCe-pJqTarAbU4Y='


## Public Key Encryption

Implementation of ElGamal encryption scheme.

In [51]:
###############################################
# IMPLEMENTATION OF ELGAMAL ENCRYPTION SCHEME #
###############################################

def elgamal_generate_keys():
    """
    Generate public and private keys for the ElGamal encryption scheme.
    The naming scheme follows what has been used in the course slides.
    """

    # generate random prime that defines the group
    p = get_random_prime()

    # find a generator for the group
    g = find_generator(p)

    # generate a private key as a random number
    x = random.randint(1, p - 1)

    # calculate the public key as g^x mod p
    h = pow(g, x, p)

    return (g, p, h), x

def elgamal_encrypt(msg, public_key):
    """
    Encrypt a message with the ElGamal encryption scheme.
    """

    # unpack public key
    g, p, h = public_key

    # generate a random number
    r = random.randint(1, p - 1)

    # calculate the two powers of the ciphertext
    gr = pow(g, r, p)
    hr = pow(h, r, p)

    # calculate the ciphertext
    c = (gr, hr * msg % p)

    return c

def elgamal_decrypt(c, private_key, public_key):
    """
    Decrypt a message with the ElGamal encryption scheme.
    """

    # unpack private key
    x = private_key

    # unpack public key
    g, p, h = public_key

    # unpack ciphertext
    c1, c2 = c

    # decrypt message
    m = (c2 * pow(c1, p - 1 - x, p)) % p

    return m

In [53]:
#####################################
# TEST OF ELGAMAL ENCRYPTION SCHEME #
#####################################

# generate keys
public_key, private_key = elgamal_generate_keys()

str_message = b"Hello"
int_message = int.from_bytes(str_message, "big")

# encrypt message
ciphertext = elgamal_encrypt(int_message, public_key)

# decrypt message
int_retrieved = elgamal_decrypt(ciphertext, private_key, public_key)

# convert the integer back to a byte string
str_retrieved = int_retrieved.to_bytes((int_retrieved.bit_length() + 7) // 8, "big")

print("Original message string:", str_message)
print("Original message integer:", int_message)
print("Encrypted message:", ciphertext)
print("Decrypted message integer:", int_retrieved)
print("Decrypted message string:", str_retrieved)

Original message string: b'Hello'
Original message integer: 310939249775
Encrypted message: (1065225926968978035, 6816520760266536516)
Decrypted message integer: 310939249775
Decrypted message string: b'Hello'


## Socket Communication

In [76]:
class Socket:
    def __init__(self, socket_type, address):
        """
        Initialize a ZeroMQ socket.
        socket_type can be zmq.REQ, zmq.REP, zmq.PUB, zmq.SUB
        """

        # init socket settings
        self.context = zmq.Context.instance()
        self.socket = self.context.socket(socket_type)
        self.address = address

        if socket_type == zmq.REP:
            self.bind()
        elif socket_type == zmq.REQ:
            self.connect()
    
    def bind(self):
        """
        Bind the socket to given address.
        """

        self.socket.bind(self.address)
    
    def connect(self):
        """
        Connect the socket to a given address.
        """

        self.socket.connect(self.address)
    
    def send(self, msg):
        """
        Send a message through the socket.
        """

        self.socket.send_pyobj(msg)
    
    def receive(self):
        """
        Receive a message from the socket.
        """

        return self.socket.recv_pyobj()
    
   
    def close(self):
        """
        Close the socket.
        """

        self.socket.close()
    
    def __del__(self):
        """
        Destructor to close the socket.
        """

        self.close()

In [77]:
server = Socket(socket_type=zmq.REP, address=f"tcp://{SERVER_HOST}:{SERVER_PORT}")
client = Socket(socket_type=zmq.REQ, address=f"tcp://{SERVER_HOST}:{SERVER_PORT}")

client.send("Hello, Server!")
print("Client received:", server.receive())

server.send("Hello, Client!")
print("Server received:", client.receive())

del server
del client

Client received: Hello, Server!
Server received: Hello, Client!


# Oblivious Transfer - Passive Security

Implementation of a 1-out-of-2 Oblivious Transfer protocol with passive security.